Proyecto jiji

In [8]:
!pip install kagglehub


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import regex as re

[nltk_data] Downloading package punkt to C:\Users\Ce
[nltk_data]     mar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
stemmer = SnowballStemmer('english')
import os
import pandas as pd

In [11]:
with open("stopwords-en.txt", encoding="latin1") as file:
   stoplist = [line.rstrip().lower() for line in file]
stoplist += ['?', '-', '.', ':', ',', '!', ';']

def preprocesamiento(texto, stemming=True):
  words = []
  texto = str(texto)
  texto = texto.lower()
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # tokenizar
  words = nltk.word_tokenize(texto, language='spanish')
  # filtrar stopwords
  words = [word for word in words if word not in stoplist]
  # reducir palabras (stemming)
  if stemming:
      words = [stemmer.stem(word) for word in words]
  return words


In [12]:
def OrdenarPorBloques(dir_blocks, n_blocks):
    for i in range(n_blocks):
        file_path = os.path.join(dir_blocks, 'block_{}.pkl'.format(i))
        with open(file_path, 'rb') as f:
            tuplas_ordenadas = pickle.load(f)
        tuplas_ordenadas = sorted(list(tuplas_ordenadas.items()), key=lambda x: x[0])
        # tuplas_ordenadas = [par for par in tuplas_ordenadas if not par[0].isdigit()]
        tuplas_ordenadas = [(term[0], list(term[1].items())) for term in tuplas_ordenadas]
        with open(file_path, 'wb') as f:
            pickle.dump(tuplas_ordenadas, f)

def mergeSortAux(dir_bloques, l, r):
    if l == r:
        bloque = leer_bloque(dir_bloques, l)
        unicos = set()    
        for par in bloque:
            unicos.add(par[0])
        return list(unicos)
    
    if (l < r):
        mid = int(math.ceil((r + l)/2.0))
        unique_l = mergeSortAux(dir_bloques, l, mid - 1)
        unique_r = mergeSortAux(dir_bloques, mid, r)
        unicos = set()    
        for term in unique_l:
            unicos.add(term)
        for term in unique_r:
            unicos.add(term)
        unicos = list(unicos)
        merge_v2(dir_bloques, l, r, mid, len(unicos))
        return list(unicos)
        
    return []

def escribir_bloque(dir_bloques, block, idx_insert_block, buffer_limit = 2000):
    with open(os.path.join(dir_bloques, "block_{}_v2.pkl".format(idx_insert_block)), 'wb') as f:
        pickle.dump(block, f)    
def leer_bloque(dir_bloques, it):
    file_path = os.path.join(dir_bloques, f"block_{it}.pkl")
    with open(file_path, "rb") as f:
        buffer = pickle.load(f)
    return buffer
def merge_v2(dir_bloques, l, r, mid, num_terms):
    idx_insert_block = l
    new_block = []
    mezclar_n_bloques = r - l + 1
    unique_terms_per_block = int(math.ceil(num_terms/mezclar_n_bloques))
    unique_terms_current_block = 0

    it_l = l
    it_r = mid
    term_dic_l = leer_bloque(dir_bloques, it_l)
    term_dic_r = leer_bloque(dir_bloques, it_r)
    
    idx_term_l = 0
    idx_term_r = 0

    idx_doc_l = 0
    idx_doc_r = 0
    new_block = []
    while(it_l < mid and it_r < r + 1):
        while(idx_term_l < len(term_dic_l) and idx_term_r < len(term_dic_r)): # moverme entre palabras de dos bloques
            new_term = []
            if(term_dic_l[idx_term_l][0] < term_dic_r[idx_term_r][0]):
                new_term = term_dic_l[idx_term_l]
                idx_term_l += 1
            elif(term_dic_l[idx_term_l][0] > term_dic_r[idx_term_r][0]):
                new_term = term_dic_r[idx_term_r]
                idx_term_r += 1
            else:
                idx_doc_l = 0
                idx_doc_r = 0
                while(idx_doc_l < len(term_dic_l[idx_term_l][1]) and idx_doc_r < len(term_dic_r[idx_term_r][1])):
                    if term_dic_l[idx_term_l][1][idx_doc_l][0] > term_dic_r[idx_term_r][1][idx_doc_r][0]:
                        pushear_doc = term_dic_r[idx_term_r][1][idx_doc_r]
                        idx_doc_r += 1
                    elif term_dic_l[idx_term_l][1][idx_doc_l][0] < term_dic_r[idx_term_r][1][idx_doc_r][0]:
                        pushear_doc = term_dic_l[idx_term_l][1][idx_doc_l]
                        idx_doc_l += 1
                    else:
                        pushear_doc = (term_dic_l[idx_term_l][1][idx_doc_l][0], term_dic_l[idx_term_l][1][idx_doc_l][1] + term_dic_r[idx_term_r][1][idx_doc_r][1])
                        idx_doc_l += 1
                        idx_doc_r += 1
                    new_term.append(pushear_doc)
                while(idx_doc_l < len(term_dic_l[idx_term_l][1])):
                    pushear_doc = term_dic_l[idx_term_l][1][idx_doc_l]
                    idx_doc_l += 1
                    new_term.append(pushear_doc)
                while(idx_doc_r < len(term_dic_r[idx_term_r][1])):
                    pushear_doc = term_dic_r[idx_term_r][1][idx_doc_r]
                    idx_doc_r += 1
                    new_term.append(pushear_doc)
                new_term = (term_dic_l[idx_term_l][0], new_term)
                idx_term_r += 1
                idx_term_l += 1
            new_block.append(new_term)
            
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
        if(len(term_dic_l) == idx_term_l):
            if (it_l < mid - 1):
                it_l += 1
                term_dic_l = leer_bloque(dir_bloques, it_l)
                idx_term_l = 0
                idx_doc_l = 0
                continue
            else:
                break
        if(len(term_dic_r) == idx_term_r):
            if (it_r < r):
                it_r += 1
                term_dic_r = leer_bloque(dir_bloques, it_r)
                idx_term_r = 0
                idx_doc_r = 0
                continue
            else:
                break
        if(it_l == mid | it_r == r + 1):
            break
    while(it_l < mid):
        term_dic_l = leer_bloque(dir_bloques, it_l)
        while(idx_term_l < len(term_dic_l)):
            new_block.append(term_dic_l[idx_term_l])
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
            idx_term_l += 1
        idx_term_l = 0
        it_l += 1
    while(it_r < r + 1):
        term_dic_r = leer_bloque(dir_bloques, it_r)
        while(idx_term_r < len(term_dic_r)):
            new_block.append(term_dic_r[idx_term_r])
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
            idx_term_r += 1
        idx_term_r = 0
        it_r += 1

    while(idx_insert_block < r + 1):
        if len(new_block) > 0:
            escribir_bloque(dir_bloques, new_block, idx_insert_block)
        else:
            escribir_bloque(dir_bloques, [], idx_insert_block)
        new_block = []
        idx_insert_block += 1
    idx_insert_block = l
    for idx_archivo in range(l, r + 1):
        nuevo_nombre = os.path.join(dir_bloques, "block_{}.pkl".format(idx_archivo))
        if os.path.exists(nuevo_nombre):
            os.remove(nuevo_nombre)
        os.rename(os.path.join(dir_bloques, "block_{}_v2.pkl".format(idx_archivo)), nuevo_nombre)
def mergeSort(dir_bloques):
    bloques_files_dir = os.listdir(os.path.join('./',dir_bloques))
    n = len(bloques_files_dir)
    mergeSortAux(dir_bloques, 0, n - 1)
'''
Recibe la direccion del folder con los diccionarios del spimi,
modifica los archivos y los sobreescribe para crear el índice
invertido con un índice global
'''
def InvertirListasDiccionarios(dir_bloques):
    n_bloques = len(os.listdir(os.path.join('./',dir_bloques)))
    for i in range(n_bloques):
        bloque_path = os.path.join(dir_bloques,"block_{}.pkl".format(i))
        with open(bloque_path, 'rb') as f:
            bloque = pickle.load(f)
        bloque_dict = {}
        if len(bloque) != 0:
            for term, poosting_list in bloque:
                poosting_dict = {}
                for doc, tf in poosting_list:
                    poosting_dict[doc] = tf
                bloque_dict[term] = poosting_dict
        with open(bloque_path, 'wb') as f:
            pickle.dump(bloque_dict, f)

def getNumberWithAtributo(dataset_head, atributo):
    if atributo in dataset_head.columns:
        return dataset_head.columns.get_loc(atributo)  # Obtiene la posición del atributo
    return -1  # Retorna -1 si no existe
 

def getResultados(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            if doc in chunk.index:
                res.append((chunk.iloc[doc], score))
    return pd.DataFrame(res, columns=['doc_id', 'score'])  
def getResultadosDF(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            row = chunk.iloc[doc].copy()
            row['score'] = score  
            res.append(row)
    return pd.DataFrame(res)  

In [18]:
import os
import pickle
import sys
import math
import dbm
import time

class SPIMI:
    def __init__(self, index_dataset_path="path",index_dir="index_blocks" , position = 3, columnas = ["track_name","track_artist","lyrics", "track_album_name"]):
        self.index_dir = index_dir  
        self.dataset_path = index_dataset_path  
        self.block_counter = 0      
        self.doc_ids = set()         
        self.idf = {}
        self.length = {}
        self.disk_limit = 4000  
        self.position = position
        
        #--Columnas:
        self.columnas = columnas 


        # ----Direciones de memoria para los files--
        self.tf_file_dir = index_dir+"_tf"
        self.idf_file_dir = index_dir+"_idf"
        self.length_file_dir = index_dir+"_lenght"
        # ----Direciones de memoria para los files--

        if not os.path.exists(self.index_dir):
            os.makedirs(self.index_dir)
    def make_tf_idf_length(self): #Despuesd e convertir todo a diccionarios 
        N = len(self.doc_ids)  
        files = sorted(os.listdir(self.index_dir))

        #Creación de los files: 
        if not os.path.exists(self.tf_file_dir):
            with dbm.open(self.tf_file_dir, "c") as tf_file:
                print("Archivo de índice creado:", self.index_dir)

        if not os.path.exists(self.idf_file_dir):
            with dbm.open(self.idf_file_dir, "c") as idf_file:
                print("Archivo de IDF creado:", self.idf_file_dir)

        if not os.path.exists(self.length_file_dir):
            with dbm.open(self.length_file_dir, "c") as length_file:
                print("Archivo de longitud creado:", self.length_file_dir)

        #Iteramos: 
        tf_dict = {}
        idf_dict = {}
        length_dict = {}
        with dbm.open(self.length_file_dir, "c") as length_db: 
            with dbm.open(self.idf_file_dir, "c") as idf_db: 
                with dbm.open(self.tf_file_dir, "c") as tf_db: 
                    for file in files:
                        if file.endswith(".pkl"):
                            block = self.load_block(os.path.join(self.index_dir, file)) #Abro el bloqque
                            for term, postings in block.items():
                                doc_freq = len(postings) # Calculate N docs
                                idf_dict[term] = math.log10(len(self.doc_ids) / (1 + doc_freq))

                                if term not in tf_dict:
                                    tf_dict[term] = []
                                for doc_id, tf in postings.items(): # Guardo el tf
                                    tf_log = math.log10(1 + tf)
                                    tf_dict[term].append((doc_id, tf_log))

                                    # Calcular contribución al cuadrado de la norma del documento
                                    if doc_id not in length_dict:
                                        length_dict[doc_id] = 0
                                    length_dict[doc_id] += (tf_log * idf_dict[term]) ** 2

                                 #Guardar en tf_file
                                tf_db[term.encode()] = pickle.dumps(tf_dict[term])
                                tf_dict.clear() 

                                 #Guardar en idf_file
                                idf_db[term.encode()] = pickle.dumps(idf_dict[term])
                                idf_db.clear()

                    for doc_id, squared_sum in length_dict.items():
                        length_dict[doc_id] = math.sqrt(squared_sum)
                        
                    for doc_id, norm in length_dict.items():
                        length_db[str(doc_id).encode()] = pickle.dumps(norm)




    def spimi_invert(self):
        dictionary = {}
        doc_ids = set()
        time_spimiInvert_start = time.time()

        #Get columna indeces: 
        with open(self.dataset_path, mode='r', encoding='utf-8') as file:
            primera_linea = file.readline().strip()
        #Get number columns
        columnas = primera_linea.split(',')
        columnas_numbers  = [i for i in range(len(columnas)) if columnas[i] in self.columnas]

        for chunk in pd.read_csv(self.dataset_path,chunksize= self.disk_limit):
            for doc_id_, row in chunk.iterrows():
                preFila = [str(row.iloc[i]) for i in columnas_numbers]
                texto = ' '.join(item for item in preFila)
                words = preprocesamiento(texto)
                for text in words:
                    doc_id = doc_id_
                    doc_ids.add(doc_id)
                    token = text
                    self.doc_ids.add(doc_id)
                    if token not in dictionary:
                        dictionary[token] = {}  

                    if doc_id not in dictionary[token]:
                        dictionary[token][doc_id] = 1  
                    else:
                        dictionary[token][doc_id] += 1  

                    dictionary_size = sys.getsizeof(dictionary)
                    if dictionary_size >= self.disk_limit:
                        self.write_block_to_disk(dictionary, level=0)
                        dictionary.clear()

        if dictionary:
            self.write_block_to_disk(dictionary, level=0)  
        self.n_docs = len(doc_ids)
        time_spimiInvert = time.time()- time_spimiInvert_start
        
        
        time_bloques_start = time.time()
        OrdenarPorBloques(dir_blocks=self.index_dir, n_blocks=self.block_counter)
        time_bloques = time.time() - time_bloques_start

        time_merge_start = time.time()
        mergeSort(self.index_dir)
        time_merge = time.time() -time_merge_start  

        time_invertir_start = time.time()
        InvertirListasDiccionarios(self.index_dir)
        time_invertir = time.time() -time_invertir_start  
        
        time_tf_idf_length_start = time.time()
        self.make_tf_idf_length()
        time_tf_idf_length = time.time() - time_tf_idf_length_start


        print("\nResumen de tiempos:")
        print(f"Tiempo para SpimiInvert: {time_spimiInvert:.2f} segundos")
        print(f"Tiempo para OrdenarPorBloques: {time_bloques:.2f} segundos")
        print(f"Tiempo para mergeSort: {time_merge:.2f} segundos")
        print(f"Tiempo para InvertirListasDiccionarios: {time_invertir:.2f} segundos")
        print(f"Tiempo para Make_tf_idf_length: {time_tf_idf_length:.2f} segundos")
        print(f"Tiempo total de creación del índice: {time_bloques + time_merge + time_invertir+  time_tf_idf_length :.2f} segundos")
        
    def write_block_to_disk(self, dictionary, level):
        sorted_terms = dict(sorted(dictionary.items())) 
        file_path = os.path.join(self.index_dir, f"block_{self.block_counter}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(sorted_terms, f)
        
        self.block_counter += 1

    def load_block(self, filepath):
        with open(filepath, "rb") as f:
            return pickle.load(f)

    
    def retrieval(self, query, k):
        N = len(self.doc_ids)  
        scores = [0] * N  
        tf_query = {}  
        terms = preprocesamiento(query)  

        # Calcular el TF-IDF del query
        for term in terms:
            if term in tf_query:
                tf_query[term] += 1
            else:
                tf_query[term] = 1

        tfidf_query = {}
        with dbm.open(self.idf_file_dir, "r") as idf_db:
            for term, tf in tf_query.items():
                idf_encoded = idf_db.get(term.encode())
                if idf_encoded:
                    idf = pickle.loads(idf_encoded)
                    tfidf_query[term] = math.log10(1 + tf) * idf
        norm_query = math.sqrt(sum(w_tq**2 for w_tq in tfidf_query.values()))

        # similitud de coseno
        with dbm.open(self.tf_file_dir, "r") as tf_db:
            with dbm.open(self.length_file_dir, "r") as length_db:
                for term, w_tq in tfidf_query.items():
                    tf_encoded = tf_db.get(term.encode())
                    if tf_encoded:
                        postings = pickle.loads(tf_encoded)  # Lista de (doc_id, tf)
                        for doc_id, tf_log in postings:
                            # Multiplicamos el tf por el idf y luego por la consulta tf-idf
                            idf_encoded = idf_db.get(term.encode())  # Obtener IDF del término
                            if idf_encoded:
                                idf0 = pickle.loads(idf_encoded)
                                tfidf_td = tf_log * idf  # TF * IDF del término en el documento
                                scores[doc_id] += tfidf_td * w_tq  # Producto punto
        # Normalizar las puntuaciones de los documentos

                for doc_id in range(N):
                    norm_encoded = length_db.get(str(doc_id).encode())

                    if norm_encoded:
                        doc_norm = pickle.loads(norm_encoded)
                        
                        if doc_norm != 0:
                            print("norm_query es: ", norm_query)
                            scores[doc_id] /= (doc_norm * norm_query)

        # Ordenar las puntuaciones en orden descendente
        result = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)

        # Devolver los k documentos más relevantes (top-k)
        return result[:k]

    

In [19]:


# ----------Set Atributo--------
# columna = getNumberWithAtributo(dataset, "lyrics")
path = "spotify_20.csv"
# ---------Creation---------
s = SPIMI( path)  
s.spimi_invert() 

# ---------End Creation---------


#
# query = get_dfText(songs, 3, 0, s.disk_limit)
query = "this is my breath"
# query = get_dfTex_Cols(songs, 2, [1,2,3,6])
top_k = 5

result = s.retrieval(query, top_k)
respuesta = getResultados(result, s.dataset_path, s.disk_limit) # La respuesta es un array pd de [fila del dataframe, score]
#-----------End of query------------


respuestadf = getResultadosDF(result, s.dataset_path, s.disk_limit)

print(respuestadf["track_name"], respuestadf["score"])



Archivo de índice creado: index_blocks
Archivo de IDF creado: index_blocks_idf
Archivo de longitud creado: index_blocks_lenght

Resumen de tiempos:
Tiempo para SpimiInvert: 0.43 segundos
Tiempo para OrdenarPorBloques: 0.14 segundos
Tiempo para mergeSort: 1.38 segundos
Tiempo para InvertirListasDiccionarios: 0.17 segundos
Tiempo para Make_tf_idf_length: 52.23 segundos
Tiempo total de creación del índice: 53.92 segundos
norm_query es:  0.0


ZeroDivisionError: float division by zero

In [ ]:

def verificar_termino(tf_file_dir, term):
    with dbm.open(tf_file_dir, "r") as tf_db:  
        try:
            term_key = term.encode()
            postings_list = pickle.loads(tf_db[term_key])
            print(f"Postings list para '{term}': {postings_list}")
        except KeyError:
            print(f"El término '{term}' no se encuentra en el archivo DBM.")

# verificar_termino(s.tf_file_dir, "6") 

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}